In [0]:
%sql
CREATE DATABASE IF NOT EXISTS silver;

CREATE TABLE IF NOT EXISTS silver.questions
USING DELTA
LOCATION '/mnt/silver/questions';

In [0]:
from pyspark.sql import functions as F

bronze_df = spark.table("bronze.questions")

silver_df = bronze_df.select(
    F.col("question_id"),
    F.from_unixtime(F.col("creation_date")).cast("timestamp").alias("creation_timestamp"),
    F.col("creation_period"),
    F.from_unixtime(F.col("last_activity_date")).cast("timestamp").alias("last_activity_timestamp"),
    F.col("tags"),
    F.col("answer_count").cast("int"),
    F.col("is_answered").cast("boolean"),
    F.col("view_count").cast("int")
).filter(
    F.col("question_id").isNotNull()
).dropDuplicates(["question_id"])

silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("creation_period") \
    .saveAsTable("silver.questions")

In [0]:
%sql SELECT * FROM silver.questions LIMIT 20